In [2]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')


In [192]:
M = np.matrix([[5,3+7j],[3-7j,10]])

In [193]:
M

matrix([[ 5.+0.j,  3.+7.j],
        [ 3.-7.j, 10.+0.j]])

In [194]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [195]:
print(H)

(7.5) [I0]
+ (3.0) [X0]
+ (-7.0) [Y0]
+ (-2.5) [Z0]


In [196]:
a = 7.5
b = 3
c = -7
d = -2.5

In [135]:
backend_state = Aer.get_backend('statevector_simulator')

In [136]:
qc = QuantumCircuit(1)
qc.u3(1,2,3,0)


result = execute(qc,backend_state).result()
out_state = result.get_statevector()
out_state

array([ 0.87758256+0.j        , -0.19951142+0.43594041j])

In [13]:
qc.draw()

┌───────────┐
q_0: ┤ U3(1,2,3) ├
     └───────────┘

In [186]:
num_shots = 100000

In [95]:
def expt_z(qc):
    
    qc1 = qc.copy()
    qc1.measure_all()
    
    result = execute(qc1, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(qc1)
    #print(counts)
    if '1' not in counts:
        counts['1'] = 0
    if '0' not in counts:
        counts['0'] = 0
    z_expect = (counts['0']- counts['1'])/(counts['0']+ counts['1'])
    
    return(z_expect)
    

In [96]:
expt_z(qc)

0.5412

In [97]:
def expt_x(qc):
    
    qc1 = qc.copy()
    # we have a state |psi> here 
    
    qc1.h(0)
    #now its been transformend to |psi'>
    
    
    qc1.measure_all()
    
    result = execute(qc1, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(qc1)
    #print(counts)
    
    if '1' not in counts:
        counts['1'] = 0
    if '0' not in counts:
        counts['0'] = 0
    x_expect = (counts['0']- counts['1'])/(counts['0']+ counts['1'])
    
    return(x_expect)

In [98]:
expt_x(qc)

-0.3498

In [101]:
def expt_y(qc):
    
    qc1 = qc.copy()
    # we have state |psi>
    qc1.rz(-pi/2,0)
    qc1.h(0)
    
    # Now its in state |psi''>
    
    
    qc1.measure_all()

    
    result = execute(qc1, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(qc1)
    #print(counts)
    
    if '1' not in counts:
        counts['1'] = 0
    if '0' not in counts:
        counts['0'] = 0
    y_expect = (counts['0']- counts['1'])/(counts['0']+ counts['1'])
    
    return(y_expect)

In [102]:
expt_y(qc)

0.759

In [103]:
E_M = 1*1+2*expt_x(qc)

In [104]:
E_M

0.27

In [105]:
psi = [ 0.87758256+0.j        , -0.19951142+0.43594041j]

In [106]:
psi_t = [[ 0.87758256+0.j        ],[ -0.19951142-0.43594041j]]

In [107]:
M

matrix([[1, 2],
        [2, 1]])

In [108]:
np.dot(psi,np.dot(M,psi_t))

matrix([[0.29964903+1.11022302e-16j]])

In [137]:
def e_m(qc):
    val = a+b*expt_x(qc)+c*expt_y(qc)+d*expt_z(qc)
    return(val)

In [138]:
e_m(qc)

-1.1978

In [139]:
params = [0,0,0]

In [140]:
def rand_psi(param):
    qc = QuantumCircuit(1)
    qc.u3(param[0],param[1],param[2],0)
    return(qc)

In [141]:
# we got varying |psi>
# we got what to minimize
# we need the algo

In [197]:
def cost(params):
    val = a+b*expt_x( rand_psi(params))+c*expt_y(rand_psi(params))+d*expt_z(rand_psi(params))
    return(val)
    

In [198]:
cost([0,0,0])

4.97952

In [199]:
from qiskit.aqua.components.optimizers import COBYLA

optimizer = COBYLA(50,0.01)

In [200]:
ret = optimizer.optimize(num_vars=3, objective_function=cost, initial_point=params)

In [201]:
ret

(array([1.25561834, 1.96104797, 0.92832713]), -0.5031400000000004, 49)

In [175]:
param_lest_cost = ret[0]

In [203]:
cost(ret[0])

-0.5305100000000008